In [1]:
!dir data

 驱动器 C 中的卷没有标签。
 卷的序列号是 0006-FDC5

 C:\Users\dell\dockerdata\乘用车细分市场销量预测\data 的目录

2019/09/26  10:55    <DIR>          .
2019/09/26  10:55    <DIR>          ..
2019/09/11  13:30           237,991 evaluation_public.csv
2019/09/19  14:39            51,762 mean_predicting_data.csv
2019/09/26  10:54        11,405,250 month_year_train.npz
2019/09/25  17:53            51,255 rf_prediction_default.csv
2019/09/11  13:31            51,712 submit_example.csv
2019/09/20  16:14            51,711 svr_predicting_sales.csv
2019/08/23  17:36           369,997 Train.zip
2019/09/12  15:33         1,545,277 train_sales_data.csv
2019/09/12  15:33         1,399,503 train_search_data.csv
2019/09/12  15:33            47,975 train_user_reply_data.csv
              10 个文件     15,212,433 字节
               2 个目录 47,247,384,576 可用字节


In [2]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [3]:
train_sales_data=pd.read_csv('data/train_sales_data.csv')
train_sales_data.head()

,province,adcode,model,bodyType,regYear,regMonth,salesVolume
0,上海,310000,3c974920a76ac9c1,SUV,2016,1,292
1,云南,530000,3c974920a76ac9c1,SUV,2016,1,466
2,内蒙古,150000,3c974920a76ac9c1,SUV,2016,1,257
3,北京,110000,3c974920a76ac9c1,SUV,2016,1,408
4,四川,510000,3c974920a76ac9c1,SUV,2016,1,610


In [4]:
model_bodytype=train_sales_data[['model','bodyType']].drop_duplicates()

In [5]:
evaluation_public=pd.read_csv('data/evaluation_public.csv')
evaluation_public.head()

,id,province,adcode,model,regYear,regMonth,forecastVolum
0,1,上海,310000,3c974920a76ac9c1,2018,1,NaN
1,2,云南,530000,3c974920a76ac9c1,2018,1,NaN
2,3,内蒙古,150000,3c974920a76ac9c1,2018,1,NaN
3,4,北京,110000,3c974920a76ac9c1,2018,1,NaN
4,5,四川,510000,3c974920a76ac9c1,2018,1,NaN


In [6]:
test_sales_data=pd.merge(evaluation_public,model_bodytype,how='left',on='model')
test_sales_data=test_sales_data.reindex(columns=['id','province','adcode','model','bodyType','regYear','regMonth'])
test_sales_data.head()

,id,province,adcode,model,bodyType,regYear,regMonth
0,1,上海,310000,3c974920a76ac9c1,SUV,2018,1
1,2,云南,530000,3c974920a76ac9c1,SUV,2018,1
2,3,内蒙古,150000,3c974920a76ac9c1,SUV,2018,1
3,4,北京,110000,3c974920a76ac9c1,SUV,2018,1
4,5,四川,510000,3c974920a76ac9c1,SUV,2018,1


In [7]:
# 生成 datetime 列
def generate_date(df):
    f=lambda x:str(x) if x >9 else '0'+str(x)
    df['date']=df['regYear'].map(lambda x:str(x))+'-'+df['regMonth'].map(f)
    df['date']=pd.to_datetime(df['date'],format='%Y-%m')
    return df

# 日期偏移 month
def month_offset_date(df,n=1):
    from pandas.tseries.offsets import DateOffset
    df['offdate']=df['date']+DateOffset(months=n)
    return df

# 日期偏移 year
def year_offset_date(df,n=1):
    from pandas.tseries.offsets import DateOffset
    df['offdate']=df['date']+DateOffset(years=n)
    return df

In [8]:
#merge data month before
def merge_month_before(df,spdf,cols=None):
    '''
    input:
        df(pd.DataFrame)--left merging data df
        spdf(pd.DataFrame)--right merging data df
    output:
        mergedf(pd.DataFrame)--merged data df
    '''
    
    df=generate_date(df)
    spdf=generate_date(spdf)
    spdf=month_offset_date(spdf)
    mergedf=pd.merge(df,spdf,how='left',left_on=['province','model','date'],right_on=['province','model','offdate'],\
                    suffixes=('','_month_before'))
    #cols=['id','province','adcode','model','regYear','regMonth','salesVolume_month_before']
    #return mergedf
    return mergedf

# merge data on the month of year before
def merge_year_before(df,spdf,cols=None):
    '''
    input:
        df(pd.DataFrame)-the left merging data df
        psdf(pd.DataFrame)-the right merging data df
    putput:
        mergeddf-the left merged data df
    '''
    df=generate_date(df)
    spdf=generate_date(spdf)
    spdf=year_offset_date(spdf,n=1)
    #onlist=['province','model','regMonth']
    mergeddf=pd.merge(df,spdf,how='left',suffixes=('','_year_before'),left_on=['province','model','date'],right_on=['province','model','offdate'])
    #cols=['province','adcode','model','bodyType','regYear','regMonth','salesVolume_year_before','salesVolume']
    return mergeddf

In [9]:
# year before merged
test_year_merged_data=merge_year_before(test_sales_data.copy(),train_sales_data)
test_year_merged_data_cols=['id','province','adcode','model','bodyType','regYear','regMonth','salesVolume']
test_year_merged_data=test_year_merged_data.reindex(columns=test_year_merged_data_cols)
test_year_merged_data.columns=['id','province','adcode','model','bodyType','regYear','regMonth','salesVolume_year_before']

In [10]:
# test_data merge 
def test_month_before_merge(df,spdf,month=1,cols=None):
    test_cols=['id','province','adcode','model','bodyType','regYear','regMonth','salesVolume','salesVolume_year_before']
    test_merge_data=merge_month_before(df,spdf)
    test_merge_data=test_merge_data.reindex(columns=test_cols)
    test_cols[-2]='salesVolume_month_before'
    test_merge_data.columns=test_cols
    month_index=test_merge_data['regMonth']==month
    return test_merge_data[month_index]

In [13]:
#model
import lightgbm as lgb
from joblib import dump,load
model=lgb.Booster(model_file='joblib/gbdt_model.txt')
ohe=load('joblib/onehotencoder.joblib')

In [14]:
# data transformation
feature_x_cat=['province','model','bodyType']
feature_x_num=['regMonth','salesVolume_month_before','salesVolume_year_before']
def test_x_array(df):
    x1=ohe.transform(df[feature_x_cat]).toarray()
    x2=df[feature_x_num]
    return np.concatenate((x1,x2),axis=1)    

In [15]:
# model predict
def model_predict(df):
    x=test_x_array(df)
    y_pred=model.predict(x)
    df['salesVolume']=np.rint(y_pred)
    return df

In [16]:
# jan data
test_data_jan=test_month_before_merge(test_year_merged_data.copy(),train_sales_data,month=1)
test_pred_jan=model_predict(test_data_jan)

In [17]:
# feb data
test_data_feb=test_month_before_merge(test_year_merged_data.copy(),test_pred_jan,month=2)
test_pred_feb=model_predict(test_data_feb)

In [18]:
# mar data
test_data_mar=test_month_before_merge(test_year_merged_data.copy(),test_pred_feb,month=3)
test_pred_mar=model_predict(test_data_mar)

In [19]:
# april data
test_data_apr=test_month_before_merge(test_year_merged_data.copy(),test_pred_mar,month=4)
test_pred_apr=model_predict(test_data_apr)

In [20]:
test_pred=pd.concat([test_pred_jan,test_pred_feb,test_pred_mar,test_pred_apr],axis=0)
test_pred.head()

,adcode,bodyType,date,id,model,offdate,province,regMonth,regYear,salesVolume,salesVolume_month_before,salesVolume_year_before
0,310000,SUV,2018-01-01,1,3c974920a76ac9c1,2018-02-01,上海,1,2018,237.0,312.0,305
1,530000,SUV,2018-01-01,2,3c974920a76ac9c1,2018-02-01,云南,1,2018,323.0,331.0,434
2,150000,SUV,2018-01-01,3,3c974920a76ac9c1,2018-02-01,内蒙古,1,2018,157.0,173.0,180
3,110000,SUV,2018-01-01,4,3c974920a76ac9c1,2018-02-01,北京,1,2018,200.0,275.0,468
4,510000,SUV,2018-01-01,5,3c974920a76ac9c1,2018-02-01,四川,1,2018,392.0,456.0,500


In [21]:
test_pred['forecastVolum']=test_pred['salesVolume'].map(lambda x:int(x))
test_pred[['id','forecastVolum']].to_csv('data/gbdt_prediction_default.csv',index=False)